In [13]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, classification_report,
    balanced_accuracy_score, roc_auc_score, confusion_matrix, precision_score,
    recall_score, f1_score
)
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results_dir = 'results/cyberbert'
os.makedirs(results_dir, exist_ok=True)
device

device(type='cuda')

In [14]:
fold_num = 2
train_data = pd.read_csv(f'data_splits/splits/fold_{fold_num}_train.csv')
test_data = pd.read_csv(f'data_splits/splits/fold_{fold_num}_test.csv')

train_data = train_data.replace({'f': 0, 't': 1})
test_data = test_data.replace({'f': 0, 't': 1})

data = pd.concat([train_data, test_data], ignore_index=True)
train_idx = pd.Series([True] * len(train_data) + [False] * len(test_data))
test_idx = pd.Series([False] * len(train_data) + [True] * len(test_data))

print(f"Using fold {fold_num} - Train: {len(train_data)}, Test: {len(test_data)}, Total: {len(data)}")
data.shape

Using fold 2 - Train: 85292, Test: 21326, Total: 106618


(106618, 63)

In [15]:
def get_target(data):
    return data['c_cyberbullying_majority'].astype(int)

def create_severity_labels(data):
    severity_cols = ['c_severity_mild_count', 'c_severity_moderate_count', 'c_severity_severe_count']
    severity_counts = data[severity_cols].fillna(0)
    cb_mask = data['c_cyberbullying_majority'] == 1
    severity_labels = np.zeros(len(data))
    
    for idx in data.index:
        if not cb_mask.iloc[idx]:
            severity_labels[idx] = 0
        else:
            row = severity_counts.iloc[idx]
            if row.sum() == 0:
                severity_labels[idx] = 0
            else:
                max_idx = row.argmax()
                severity_labels[idx] = max_idx + 1
    return severity_labels.astype(int)

def create_topic_labels(data):
    topic_cols = ['c_topic_disability_majority', 'c_topic_gender_majority', 'c_topic_intellectual_majority',
                  'c_topic_other_majority', 'c_topic_physical_majority', 'c_topic_political_majority',
                  'c_topic_race_majority', 'c_topic_religious_majority', 'c_topic_sexual_majority',
                  'c_topic_social_status_majority']
    topic_data = data[topic_cols].fillna(0).astype(int)
    cb_mask = data['c_cyberbullying_majority'] == 1
    topic_labels = topic_data.copy()
    topic_labels[~cb_mask] = 0
    return topic_labels, topic_cols

def create_role_labels(data):
    cb_roles = ['c_role_bully_count', 'c_role_cb__bully_assistant_count', 
                'c_role_cb_aggressive_victim_role_count', 'c_role_cb_aggressive_defender_count']
    noncb_roles = ['c_role_noncb_passive_bystander_count', 'c_role_noncb_non_aggressive_victim_count',
                   'c_role_noncb_non_aggressive_defender_count']
    role_labels = []
    cb_mask = data['c_cyberbullying_majority'] == 1
    
    for idx in data.index:
        if cb_mask.iloc[idx]:
            role_counts = data[cb_roles].iloc[idx]
            if role_counts.sum() > 0:
                max_role = role_counts.argmax()
                role_labels.append(max_role)
            else:
                role_labels.append(0)
        else:
            role_counts = data[noncb_roles].iloc[idx]
            if role_counts.sum() > 0:
                max_role = role_counts.argmax()
                role_labels.append(max_role + 4)
            else:
                role_labels.append(4)
    return np.array(role_labels)

def save_metrics(y_true, y_pred, y_proba, class_names, task_name, model_name='cyberbert'):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    metrics = {}
    
    metrics['accuracy'] = accuracy_score(y_true, y_pred)
    metrics['balanced_accuracy'] = balanced_accuracy_score(y_true, y_pred)
    
    metrics['precision_macro'] = precision_score(y_true, y_pred, average='macro', zero_division=0)
    metrics['recall_macro'] = recall_score(y_true, y_pred, average='macro', zero_division=0)
    metrics['f1_macro'] = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    metrics['precision_weighted'] = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    metrics['recall_weighted'] = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    metrics['f1_weighted'] = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    metrics['precision_per_class'] = precision_score(y_true, y_pred, average=None, zero_division=0).tolist()
    metrics['recall_per_class'] = recall_score(y_true, y_pred, average=None, zero_division=0).tolist()
    metrics['f1_per_class'] = f1_score(y_true, y_pred, average=None, zero_division=0).tolist()
    
    if len(np.unique(y_true)) == 2:
        if y_proba is not None and y_proba.shape[1] >= 2:
            metrics['auroc'] = roc_auc_score(y_true, y_proba[:, 1])
        else:
            metrics['auroc'] = None
    else:
        if y_proba is not None:
            try:
                metrics['auroc_macro'] = roc_auc_score(y_true, y_proba, multi_class='ovr', average='macro')
                metrics['auroc_weighted'] = roc_auc_score(y_true, y_proba, multi_class='ovr', average='weighted')
            except:
                metrics['auroc_macro'] = None
                metrics['auroc_weighted'] = None
        else:
            metrics['auroc_macro'] = None
            metrics['auroc_weighted'] = None
    
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{model_name.upper()} - {task_name} - Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(f'{results_dir}/{task_name}_confusion_matrix_{timestamp}_fold_{fold_num}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    with open(f'{results_dir}/{task_name}_metrics_{timestamp}_fold_{fold_num}.json', 'w') as f:
        json.dump(metrics, f, indent=2)
    
    np.save(f'{results_dir}/{task_name}_confusion_matrix_{timestamp}_fold_{fold_num}.npy', cm)
    
    print(f"Saved metrics for {task_name} to {results_dir}")
    
    return metrics

In [16]:
target = get_target(data)
severity_labels = create_severity_labels(data)
topic_labels, topic_cols = create_topic_labels(data)
role_labels = create_role_labels(data)

text_data = data['c_comment_content'].fillna('').astype(str)

f"Data: {data.shape}, Train: {train_idx.sum()}, Test: {test_idx.sum()}"

'Data: (106618, 63), Train: 85292, Test: 21326'

In [17]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx]) if hasattr(self.texts, 'iloc') else str(self.texts[idx])
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [18]:
def freeze_base_model(model):
    for name, param in model.named_parameters():
        if 'classifier' not in name and 'pooler' not in name:
            param.requires_grad = False
    return model

def train_transformer_model(model_name, tokenizer_class, model_class, train_texts, train_labels, test_texts, test_labels, num_labels, task_name):
    tokenizer = tokenizer_class.from_pretrained(model_name)
    model = model_class.from_pretrained(
        model_name, 
        num_labels=num_labels,
        ignore_mismatched_sizes=True,
        problem_type="single_label_classification",
        classifier_dropout=0.1
    ).to(device)
    
    model = freeze_base_model(model)
    
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)
    
    training_args = TrainingArguments(
        output_dir='./results_temp',
        num_train_epochs=20,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        learning_rate=5e-4,
        logging_dir='./logs',
        logging_steps=10,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        report_to=None
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )
    
    trainer.train()
    
    predictions = trainer.predict(test_dataset)
    pred_probs = torch.softmax(torch.tensor(predictions.predictions), dim=1).numpy()
    pred_labels = np.argmax(predictions.predictions, axis=1)
    
    del model, trainer
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    return pred_labels, pred_probs

In [19]:
print("BINARY CYBERBULLYING CLASSIFICATION:")
print("=" * 50)

train_texts = text_data[train_idx]
test_texts = text_data[test_idx]
train_labels = target[train_idx].values
test_labels = target[test_idx].values

pred, pred_probs = train_transformer_model(
    'unitary/toxic-bert',
    BertTokenizer,
    BertForSequenceClassification,
    train_texts,
    train_labels,
    test_texts,
    test_labels,
    num_labels=2,
    task_name='binary_classification'
)

metrics = save_metrics(
    test_labels, pred, pred_probs, 
    ['Non-CB', 'CB'], 
    'binary_classification'
)

print(f"CyberBERT: Acc={metrics['accuracy']:.3f}, Balanced Acc={metrics['balanced_accuracy']:.3f}, F1={metrics['f1_weighted']:.3f}, AUROC={metrics['auroc']:.3f}")
print(classification_report(test_labels, pred, target_names=['Non-CB', 'CB']))

BINARY CYBERBULLYING CLASSIFICATION:


Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/home/jsunayana/miniconda3/envs/privatefl2/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/jsunayana/miniconda3/envs/privatefl2/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jsunayana/miniconda3/envs/privatefl2/lib/python3.8/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/jsunayana/miniconda3/envs/privatefl2/lib/python3.8/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/home/jsunayana/miniconda3/envs/privatefl2/lib/python3.8/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'
Some weights of BertForSequenceClassification were not initialized from 

  0%|          | 0/106620 [00:00<?, ?it/s]

{'loss': 0.619, 'grad_norm': 10.551852226257324, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 0.5359, 'grad_norm': 10.227927207946777, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 0.4233, 'grad_norm': 3.2449898719787598, 'learning_rate': 3e-05, 'epoch': 0.01}
{'loss': 0.2803, 'grad_norm': 3.181812286376953, 'learning_rate': 4e-05, 'epoch': 0.01}
{'loss': 0.3139, 'grad_norm': 1.503793478012085, 'learning_rate': 5e-05, 'epoch': 0.01}
{'loss': 0.2831, 'grad_norm': 1.4028441905975342, 'learning_rate': 6e-05, 'epoch': 0.01}
{'loss': 0.2669, 'grad_norm': 4.312163352966309, 'learning_rate': 7.000000000000001e-05, 'epoch': 0.01}
{'loss': 0.2855, 'grad_norm': 0.9242307543754578, 'learning_rate': 8e-05, 'epoch': 0.02}
{'loss': 0.2192, 'grad_norm': 2.0670363903045654, 'learning_rate': 8.999999999999999e-05, 'epoch': 0.02}
{'loss': 0.1842, 'grad_norm': 0.8820130228996277, 'learning_rate': 0.0001, 'epoch': 0.02}
{'loss': 0.211, 'grad_norm': 1.368399977684021, 'learning_rate': 0.00011, 'epoch': 

  0%|          | 0/1333 [00:00<?, ?it/s]

...

{'eval_loss': 0.1884542554616928, 'eval_runtime': 430.8975, 'eval_samples_per_second': 49.492, 'eval_steps_per_second': 3.094, 'epoch': 20.0}


There were missing keys in the checkpoint model loaded: ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.0.output.La

{'train_runtime': 52906.5228, 'train_samples_per_second': 32.243, 'train_steps_per_second': 2.015, 'train_loss': 0.19677243474622846, 'epoch': 20.0}


  0%|          | 0/1333 [00:00<?, ?it/s]

Saved metrics for binary_classification to results/cyberbert
CyberBERT: Acc=0.927, Balanced Acc=0.769, F1=0.923, AUROC=0.938
              precision    recall  f1-score   support

      Non-CB       0.95      0.97      0.96     18927
          CB       0.73      0.56      0.64      2399

    accuracy                           0.93     21326
   macro avg       0.84      0.77      0.80     21326
weighted avg       0.92      0.93      0.92     21326



In [20]:
print("\nSEVERITY CLASSIFICATION (CYBERBULLYING COMMENTS ONLY):")
print("=" * 60)

cb_mask = target == 1
severity_names = ['mild', 'moderate', 'severe']

train_texts_sev = text_data[train_idx & cb_mask]
test_texts_sev = text_data[test_idx & cb_mask]
train_labels_sev = severity_labels[train_idx & cb_mask] - 1  # Convert 1,2,3 to 0,1,2
test_labels_sev = severity_labels[test_idx & cb_mask] - 1    # Convert 1,2,3 to 0,1,2

# Filter out any remaining 'none' labels (originally 0, now -1)
valid_train_mask = train_labels_sev >= 0
valid_test_mask = test_labels_sev >= 0

train_texts_sev = train_texts_sev[valid_train_mask]
test_texts_sev = test_texts_sev[valid_test_mask]
train_labels_sev = train_labels_sev[valid_train_mask]
test_labels_sev = test_labels_sev[valid_test_mask]

if len(train_labels_sev) > 0 and len(test_labels_sev) > 0:
    pred_s, pred_probs_s = train_transformer_model(
        'unitary/toxic-bert',
        BertTokenizer,
        BertForSequenceClassification,
        train_texts_sev,
        train_labels_sev,
        test_texts_sev,
        test_labels_sev,
        num_labels=3,
        task_name='severity_classification'
    )
    
    metrics_s = save_metrics(
        test_labels_sev, pred_s, pred_probs_s,
        severity_names,
        'severity_classification'
    )
    
    print(f"CyberBERT: Acc={metrics_s['accuracy']:.3f}, Balanced Acc={metrics_s['balanced_accuracy']:.3f}, F1={metrics_s['f1_weighted']:.3f}")
    if metrics_s['auroc_weighted']:
        print(f"AUROC Weighted={metrics_s['auroc_weighted']:.3f}")
    print(classification_report(test_labels_sev, pred_s, target_names=severity_names, zero_division=0))
else:
    print("CyberBERT: No data available")


SEVERITY CLASSIFICATION (CYBERBULLYING COMMENTS ONLY):


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at unitary/toxic-bert and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([6]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([6, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/11100 [00:00<?, ?it/s]

{'loss': 1.0559, 'grad_norm': 7.98483419418335, 'learning_rate': 1e-05, 'epoch': 0.02}
{'loss': 0.9665, 'grad_norm': 6.100740432739258, 'learning_rate': 2e-05, 'epoch': 0.04}
{'loss': 0.8951, 'grad_norm': 4.961310386657715, 'learning_rate': 3e-05, 'epoch': 0.05}
{'loss': 0.8193, 'grad_norm': 4.548985004425049, 'learning_rate': 4e-05, 'epoch': 0.07}
{'loss': 0.7661, 'grad_norm': 4.029865741729736, 'learning_rate': 5e-05, 'epoch': 0.09}
{'loss': 0.675, 'grad_norm': 5.8566203117370605, 'learning_rate': 6e-05, 'epoch': 0.11}
{'loss': 0.7158, 'grad_norm': 4.430349349975586, 'learning_rate': 7.000000000000001e-05, 'epoch': 0.13}
{'loss': 0.6678, 'grad_norm': 3.6121349334716797, 'learning_rate': 8e-05, 'epoch': 0.14}
{'loss': 0.6869, 'grad_norm': 2.0760090351104736, 'learning_rate': 8.999999999999999e-05, 'epoch': 0.16}
{'loss': 0.7114, 'grad_norm': 5.729135990142822, 'learning_rate': 0.0001, 'epoch': 0.18}
{'loss': 0.7019, 'grad_norm': 2.2787322998046875, 'learning_rate': 0.00011, 'epoch': 0

  0%|          | 0/150 [00:00<?, ?it/s]

...

{'eval_loss': 0.6640679836273193, 'eval_runtime': 48.2959, 'eval_samples_per_second': 49.673, 'eval_steps_per_second': 3.106, 'epoch': 20.0}


There were missing keys in the checkpoint model loaded: ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.0.output.La

{'train_runtime': 5597.4643, 'train_samples_per_second': 31.682, 'train_steps_per_second': 1.983, 'train_loss': 0.6275923202703665, 'epoch': 20.0}


  0%|          | 0/150 [00:00<?, ?it/s]

Saved metrics for severity_classification to results/cyberbert
CyberBERT: Acc=0.674, Balanced Acc=0.625, F1=0.675
AUROC Weighted=0.771
              precision    recall  f1-score   support

        mild       0.75      0.75      0.75      1399
    moderate       0.57      0.58      0.57       864
      severe       0.58      0.55      0.56       136

    accuracy                           0.67      2399
   macro avg       0.63      0.63      0.63      2399
weighted avg       0.67      0.67      0.67      2399



In [21]:
print("\nROLE CLASSIFICATION (ALL COMMENTS):")
print("=" * 40)

role_names = ['bully', 'bully_assistant', 'aggressive_victim', 'aggressive_defender',
              'passive_bystander', 'non_aggressive_victim', 'non_aggressive_defender']

train_texts_role = text_data[train_idx]
test_texts_role = text_data[test_idx]
train_labels_role = role_labels[train_idx]
test_labels_role = role_labels[test_idx]

pred_r, pred_probs_r = train_transformer_model(
    'unitary/toxic-bert',
    BertTokenizer,
    BertForSequenceClassification,
    train_texts_role,
    train_labels_role,
    test_texts_role,
    test_labels_role,
    num_labels=7,
    task_name='role_classification'
)

metrics_r = save_metrics(
    test_labels_role, pred_r, pred_probs_r,
    role_names,
    'role_classification'
)

print(f"CyberBERT: Acc={metrics_r['accuracy']:.3f}, Balanced Acc={metrics_r['balanced_accuracy']:.3f}, F1={metrics_r['f1_weighted']:.3f}")
if metrics_r['auroc_weighted']:
    print(f"AUROC Weighted={metrics_r['auroc_weighted']:.3f}")
print(classification_report(test_labels_role, pred_r, target_names=role_names, zero_division=0))


ROLE CLASSIFICATION (ALL COMMENTS):


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at unitary/toxic-bert and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([6]) in the checkpoint and torch.Size([7]) in the model instantiated
- classifier.weight: found shape torch.Size([6, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/106620 [00:00<?, ?it/s]

{'loss': 1.6246, 'grad_norm': 15.7592134475708, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 1.4846, 'grad_norm': 13.806464195251465, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 1.214, 'grad_norm': 8.679006576538086, 'learning_rate': 3e-05, 'epoch': 0.01}
{'loss': 0.8746, 'grad_norm': 9.061903953552246, 'learning_rate': 4e-05, 'epoch': 0.01}
{'loss': 0.6804, 'grad_norm': 5.03314208984375, 'learning_rate': 5e-05, 'epoch': 0.01}
{'loss': 0.5577, 'grad_norm': 2.5873780250549316, 'learning_rate': 6e-05, 'epoch': 0.01}
{'loss': 0.4234, 'grad_norm': 4.096367359161377, 'learning_rate': 7.000000000000001e-05, 'epoch': 0.01}
{'loss': 0.3533, 'grad_norm': 1.251061201095581, 'learning_rate': 8e-05, 'epoch': 0.02}
{'loss': 0.3723, 'grad_norm': 2.5483028888702393, 'learning_rate': 8.999999999999999e-05, 'epoch': 0.02}
{'loss': 0.3974, 'grad_norm': 0.6858304142951965, 'learning_rate': 0.0001, 'epoch': 0.02}
{'loss': 0.4349, 'grad_norm': 1.4728114604949951, 'learning_rate': 0.00011, 'epoch': 0.0

  0%|          | 0/1333 [00:00<?, ?it/s]

...

{'eval_loss': 0.32862788438796997, 'eval_runtime': 429.2397, 'eval_samples_per_second': 49.683, 'eval_steps_per_second': 3.105, 'epoch': 19.0}
{'loss': 0.4218, 'grad_norm': 3.9452998638153076, 'learning_rate': 2.51130795326046e-05, 'epoch': 19.0}
{'loss': 0.4749, 'grad_norm': 3.245302438735962, 'learning_rate': 2.5065963060686017e-05, 'epoch': 19.0}
{'loss': 0.2728, 'grad_norm': 3.0066728591918945, 'learning_rate': 2.5018846588767432e-05, 'epoch': 19.0}
{'loss': 0.2274, 'grad_norm': 0.843357264995575, 'learning_rate': 2.497173011684885e-05, 'epoch': 19.01}
{'loss': 0.3267, 'grad_norm': 1.678183674812317, 'learning_rate': 2.4924613644930267e-05, 'epoch': 19.01}
{'loss': 0.2885, 'grad_norm': 2.070295810699463, 'learning_rate': 2.4877497173011682e-05, 'epoch': 19.01}
{'loss': 0.2107, 'grad_norm': 1.2405740022659302, 'learning_rate': 2.4830380701093105e-05, 'epoch': 19.01}
{'loss': 0.358, 'grad_norm': 1.9432109594345093, 'learning_rate': 2.478326422917452e-05, 'epoch': 19.01}
{'loss': 0.24

  0%|          | 0/1333 [00:00<?, ?it/s]

{'eval_loss': 0.32767337560653687, 'eval_runtime': 429.0304, 'eval_samples_per_second': 49.707, 'eval_steps_per_second': 3.107, 'epoch': 20.0}
{'train_runtime': 52901.455, 'train_samples_per_second': 32.246, 'train_steps_per_second': 2.015, 'train_loss': 0.33238066860623666, 'epoch': 20.0}


  0%|          | 0/1333 [00:00<?, ?it/s]

Saved metrics for role_classification to results/cyberbert
CyberBERT: Acc=0.899, Balanced Acc=0.220, F1=0.881
AUROC Weighted=0.921
                         precision    recall  f1-score   support

                  bully       0.63      0.54      0.58      2036
        bully_assistant       0.00      0.00      0.00        39
      aggressive_victim       0.00      0.00      0.00        74
    aggressive_defender       0.08      0.00      0.01       250
      passive_bystander       0.93      0.98      0.95     18512
  non_aggressive_victim       0.00      0.00      0.00       256
non_aggressive_defender       0.06      0.01      0.02       159

               accuracy                           0.90     21326
              macro avg       0.24      0.22      0.22     21326
           weighted avg       0.86      0.90      0.88     21326



In [ ]:
class MultiLabelTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx]) if hasattr(self.texts, 'iloc') else str(self.texts[idx])
        text = text.strip() if text and text != 'nan' else ""
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }

class MultiLabelTransformer(nn.Module):
    def __init__(self, model_name, num_labels, model_class):
        super().__init__()
        self.transformer = model_class.from_pretrained(
            model_name, 
            num_labels=num_labels,
            ignore_mismatched_sizes=True,
            problem_type="multi_label_classification"
        )
        
        for name, param in self.transformer.named_parameters():
            if 'classifier' not in name and 'pooler' not in name:
                param.requires_grad = False
        
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = self.sigmoid(logits)
        
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels.float())
            return {'loss': loss, 'logits': logits}
        
        return {'logits': logits}

def train_multilabel_transformer(model_name, tokenizer_class, model_class, train_texts, train_labels, test_texts, test_labels, num_labels):
    tokenizer = tokenizer_class.from_pretrained(model_name)
    model = MultiLabelTransformer(model_name, num_labels, model_class).to(device)
    
    train_dataset = MultiLabelTextDataset(train_texts, train_labels, tokenizer)
    test_dataset = MultiLabelTextDataset(test_texts, test_labels, tokenizer)
    
    training_args = TrainingArguments(
        output_dir='./results_temp',
        num_train_epochs=20,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        learning_rate=5e-4,
        logging_dir='./logs',
        logging_steps=500,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=False,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        report_to=None,
        save_total_limit=1,
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )
    
    print("Training multi-label model...")
    trainer.train()
    
    predictions = trainer.predict(test_dataset)
    pred_probs = torch.sigmoid(torch.tensor(predictions.predictions))
    pred_labels = (pred_probs > 0.5).numpy().astype(int)
    
    del model, trainer
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    return pred_labels, pred_probs.numpy()

def save_multilabel_metrics(y_true, y_pred, y_proba, topic_names, task_name, model_name):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    metrics = {}
    
    topic_metrics = []
    balanced_accuracies = []
    
    for i, topic_name in enumerate(topic_names):
        p, r, f1, _ = precision_recall_fscore_support(
            y_true[:, i], y_pred[:, i], average='binary', zero_division=0
        )
        support = y_true[:, i].sum()
        
        balanced_acc = balanced_accuracy_score(y_true[:, i], y_pred[:, i])
        balanced_accuracies.append(balanced_acc)
        
        try:
            auroc = roc_auc_score(y_true[:, i], y_proba[:, i])
        except:
            auroc = None
        
        cm = confusion_matrix(y_true[:, i], y_pred[:, i])
        
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
        plt.title(f'{model_name.upper()} - {topic_name} Topic - Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(f'{results_dir}/{task_name}_{topic_name}_confusion_matrix_{timestamp}_fold_{fold_num}.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        np.save(f'{results_dir}/{task_name}_{topic_name}_confusion_matrix_{timestamp}_fold_{fold_num}.npy', cm)
            
        topic_metrics.append({
            'topic': topic_name,
            'precision': p,
            'recall': r,
            'f1': f1,
            'balanced_accuracy': balanced_acc,
            'auroc': auroc,
            'support': support
        })
    
    subset_accuracy = np.mean(np.all(y_true == y_pred, axis=1))
    
    precisions = [m['precision'] for m in topic_metrics]
    recalls = [m['recall'] for m in topic_metrics]
    f1s = [m['f1'] for m in topic_metrics]
    aurocs = [m['auroc'] for m in topic_metrics if m['auroc'] is not None]
    
    metrics['subset_accuracy'] = subset_accuracy
    metrics['macro_precision'] = np.mean(precisions)
    metrics['macro_recall'] = np.mean(recalls)
    metrics['macro_f1'] = np.mean(f1s)
    metrics['macro_balanced_accuracy'] = np.mean(balanced_accuracies)
    if aurocs:
        metrics['macro_auroc'] = np.mean(aurocs)
    
    metrics['per_topic_metrics'] = topic_metrics
    
    with open(f'{results_dir}/{task_name}_metrics_{timestamp}_fold_{fold_num}.json', 'w') as f:
        json.dump(metrics, f, indent=2, default=str)
    
    print(f"Saved multi-label metrics for {task_name} to {results_dir}")
    
    return metrics

In [23]:
print("\nTOPIC CLASSIFICATION (CYBERBULLYING COMMENTS ONLY):")
print("=" * 60)

cb_mask = target == 1
topic_names = ['disability', 'gender', 'intellectual', 'other', 'physical', 
               'political', 'race', 'religious', 'sexual', 'social_status']

train_texts_topic = text_data[train_idx & cb_mask]
test_texts_topic = text_data[test_idx & cb_mask]
train_labels_topic = topic_labels[train_idx & cb_mask].values
test_labels_topic = topic_labels[test_idx & cb_mask].values

if len(train_labels_topic) > 0 and len(test_labels_topic) > 0:
    pred_t, pred_probs_t = train_multilabel_transformer(
        'unitary/toxic-bert',
        BertTokenizer,
        BertForSequenceClassification,
        train_texts_topic,
        train_labels_topic,
        test_texts_topic,
        test_labels_topic,
        num_labels=10
    )
    
    metrics_t = save_multilabel_metrics(
        test_labels_topic, pred_t, pred_probs_t,
        topic_names,
        'topic_classification',
        'cyberbert'
    )
    
    print(f"CyberBERT: Subset Acc={metrics_t['subset_accuracy']:.3f}, Macro F1={metrics_t['macro_f1']:.3f}, Macro Balanced Acc={metrics_t['macro_balanced_accuracy']:.3f}")
    if 'macro_auroc' in metrics_t:
        print(f"Macro AUROC={metrics_t['macro_auroc']:.3f}")
    
    print("\nPer-topic results:")
    for metric in metrics_t['per_topic_metrics']:
        print(f"{metric['topic']}: P={metric['precision']:.3f}, R={metric['recall']:.3f}, F1={metric['f1']:.3f}, Balanced Acc={metric['balanced_accuracy']:.3f}, Support={metric['support']}")
else:
    print("CyberBERT: No data available")


TOPIC CLASSIFICATION (CYBERBULLYING COMMENTS ONLY):


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at unitary/toxic-bert and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([6]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([6, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training multi-label model...


  0%|          | 0/11100 [00:00<?, ?it/s]

{'loss': 0.2727, 'grad_norm': 0.5264139175415039, 'learning_rate': 0.0005, 'epoch': 0.9}


...

  0%|          | 0/150 [00:00<?, ?it/s]

{'eval_loss': 0.1708691567182541, 'eval_runtime': 47.2619, 'eval_samples_per_second': 50.76, 'eval_steps_per_second': 3.174, 'epoch': 20.0}


Could not locate the best model at ./results_temp/checkpoint-9990/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


{'train_runtime': 5543.3252, 'train_samples_per_second': 31.992, 'train_steps_per_second': 2.002, 'train_loss': 0.17916240554672103, 'epoch': 20.0}


FileNotFoundError: [Errno 2] No such file or directory: './results_temp/checkpoint-9990'

In [ ]:
print("\nTOPIC CLASSIFICATION (CYBERBERT):")
print("=" * 60)

cb_mask = target == 1
topic_names = ['disability', 'gender', 'intellectual', 'other', 'physical', 
               'political', 'race', 'religious', 'sexual', 'social_status']

train_texts_topic = text_data[train_idx & cb_mask]
test_texts_topic = text_data[test_idx & cb_mask]
train_labels_topic = topic_labels[train_idx & cb_mask].values
test_labels_topic = topic_labels[test_idx & cb_mask].values

if len(train_labels_topic) > 0 and len(test_labels_topic) > 0:
    pred_t, pred_probs_t = train_multilabel_transformer(
        'unitary/toxic-bert',
        BertTokenizer,
        BertForSequenceClassification,
        train_texts_topic,
        train_labels_topic,
        test_texts_topic,
        test_labels_topic,
        num_labels=10
    )

    metrics_t = save_multilabel_metrics(
        test_labels_topic, pred_t, pred_probs_t,
        topic_names,
        'topic_classification',
        'cyberbert'
    )

    print(f"CyberBERT: Subset Acc={metrics_t['subset_accuracy']:.3f}, Macro F1={metrics_t['macro_f1']:.3f}, Macro Balanced Acc={metrics_t['macro_balanced_accuracy']:.3f}")
    if 'macro_auroc' in metrics_t:
        print(f"Macro AUROC={metrics_t['macro_auroc']:.3f}")

    print("\nPer-topic results:")
    for metric in metrics_t['per_topic_metrics']:
        print(f"{metric['topic']}: P={metric['precision']:.3f}, R={metric['recall']:.3f}, F1={metric['f1']:.3f}, Balanced Acc={metric['balanced_accuracy']:.3f}, Support={metric['support']}")
else:
    print("CyberBERT: No data available")
